<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h1 style="font-size: 32px;">Introduction</h1>
<p>
    Kleinian functions, also known as the generalised Wierstrass $\wp_{ij}$ functions, are important tools for studying hyperelliptic curves. The main objective of this notebook is to present code that can compute Kleinian functions for a hyperelliptic curve of genus 2.  While the primary focus is on the computation aspect, related concepts and background information are provided for context. 
</p>
<p>
This notebook is inspired by the work of Julia Bernatska <a href="https://arxiv.org/abs/2407.05632" target="_blank">[1]</a> and the code attached to an article posted on WolframCommunity <a href="https://community.wolfram.com/groups/-/m/t/3243472" target="_blank">[2]</a>. It also draws from the SageMath Riemann surfaces documentation <em>SageMath Riemann surfaces</em> documentation <a href="https://doc.sagemath.org/html/en/reference/curves/sage/schemes/riemann_surfaces/riemann_surface.html#sage.schemes.riemann_surfaces.riemann_surface.RiemannSurface"
target="_blank">[3]</a>. All additional references are listed in the Literature section at the end of the notebook.
</p>
</div>


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h1 style="font-size: 32px;">1. Preliminaries</h1>
<p>
    Many works on hyperelliptic functions and Riemann surfaces use similar notation, which can lead to potential naming collisions. To avoid such collisions, a legend of notations used in this notebook is provided below.
</p>
<h3 style="font-size: 24px;">1.1. Legend</h3>
    <ul>
        <li>$\mathscr{C}$ - an algebraic curve</li>
        <li>$X$ - a Riemann surface</li>
         <li>$\lambda_i$ - coefficients of the equation of $\mathscr{C}$ </li>
        <li>$e_i$ - branch points</li>
        <li>$g$ - a genus of $\mathscr{C}$</li>    
        <li>$du$ - first kind (or holomorphic) differential on $\mathscr{C}$ </li>
        <li>$dr$ - second kind differential on $\mathscr{C}$</li>
        <li>$\mathfrak{a}_i, \mathfrak{b}_i$ - not normalised canonical homology cycles</li>
        <li>$\omega$ -  first kind integrals (first kind not normalised $\mathfrak{a}$, and $\mathfrak{b}$-period matrices)</li>
        <li>$\eta$ - second kind integrals (second kind not normalised $\mathfrak{a}$, and $\mathfrak{b}$-period matrices)</li>
        <li>$\mathrm{Jac}(\mathscr{C}) = \mathbb{C}^g/\{\omega, \omega' \}$ - Jacobian variety of the curve $\mathscr{C}$</li>
        <li>$D$ - Divisor  on the Riemann surface $X$</li>
        <li>$\mathcal{A}(P)$, $\mathcal{A}(D)$  - Abel image (or first kind integral) of a point $P$ and a divisor $D$</li>
        <li>$\Sigma$ - the theta-divisor defined by $\{ \mathbf{u}\in \mathrm{Jac}(\mathscr{C})| \sigma(\mathbf{u})=0 \}$ </li>
        <li>$\mathfrak{U}(\mathscr{C})$ - differential field of $\wp$-functions on $\mathrm{Jac}(\mathscr{C}) \setminus \Sigma$</li>
        <li>$\mathbf{\varepsilon}$ - a characteristic vectors</li>
        <li>$\mathbf{K}$ - a vector of Riemann constants </li>
</ul>
  
</div>

<div>
<h3 style="font-family: 'Latin Modern Roman', Times, serif; font-size: 24px;"> 1.2. Hyperelliptic curve</h3>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
    In studies on hyperelliptic functions, various conventions for defining a curve, and consequently the hyperelliptic functions, can be found. In this notebook, we adopt the convention presented in <a href="https://arxiv.org/abs/2407.05632" target="_blank">[1]</a>. <br>
    <br>
    Additionally, conventions for curves in both general and canonical forms are also discussed in dedicated notebooks: `canonical-name.ipynb` and `general-name.ipynb`. Please refer to these notebooks for further details.
</p>
</div>
 


<div style="background-color:  rgba(224, 224, 224, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(192, 192, 192); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(160, 160, 160);">Definition 1.1.</h4>
    <p>    
        The hyperelliptic curve is defined by 
        $$
            \mathscr{C} = \{ (x,y)\in \mathbb{C}^2 \mid f(x,y)=0 \},
        $$
        where
        $$
            f(x,y) = -y^2 + x^{2g+1} + \sum_{i=0}^{2g}\lambda_{2i+2}x^{2g-i}, \quad \lambda_{k\leq0}=0,\; \lambda_k \in \mathbb{R}.$
        $$
    </p>
</div>


<div style="background-color:  rgba(153, 204, 255, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(0, 128,255); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(0, 128,255);">Note 1.1.</h4>
    <p>
    <ul>
        We assume $\lambda_k \in \mathbb{R}$, instead of $\lambda_k \in \mathbb{C}$ which can be found in various texts, because we would like to apply these functions to physical equations first.
    </ul>            
  </p>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
    So for a curve of genus $g=2$ we get
    $$
        f(x,y) = -y^2 + x^5 + \lambda_2 x^4 + \lambda_4 x^3+ \lambda_6 x^2+ \lambda_8 x+ \lambda_{10},
    $$   
    Let's load the appropriate SageMath library and define the curve $\mathscr{C}$
</p>    
</div>

In [1]:
from sage.schemes.riemann_surfaces.riemann_surface import RiemannSurface

In [2]:
# Defines the lambda coefficients
lambda2 = 12.0
lambda4 = 2.0
lambda6  = 0.9
lambda8  = 21.0
lambda10  = 10.0

<div style="background-color:  rgba(255, 153, 153, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(255, 51,51); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(255, 51,51);">Important 1.1.</h4>
    <p>
        The coefficients above must be real-floating-point numbers        
    </p>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
    Since the current Sage library only works well on the field of rational numbers, we have to approximate all function coefficients by these numbers.
</p>
</div>

In [3]:
# Rational approximation
l2 = lambda2.nearby_rational(max_error=1e-10)
l4 = lambda4.nearby_rational(max_error=1e-10)
l6 = lambda6.nearby_rational(max_error=1e-10)
l8 = lambda8.nearby_rational(max_error=1e-10)
l10 = lambda10.nearby_rational(max_error=1e-10)

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
    Next, we need to define the variables $x$ and $y$ in the ring of polynomials over the rational numbers.
</p>
</div>

In [4]:
R.<x, y> = PolynomialRing(QQ, 2)

<div style="background-color:  rgba(153, 204, 255, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(0, 128,255); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(0, 128,255);">Note 1.2.</h4>
    <p>
        The order of variables can be important depending on how you write your polynomial. Make sure you use the same order in your polynomial expression.        
    </p>
</div>

In [5]:
# Defining the polynomial f
f = -y^2 + x^5 + l2*x^4 + l4*x^3 + l6*x^2 + l8*x + l10

<div>
<h3 style="font-family: 'Latin Modern Roman', Times, serif; font-size: 24px;">1.3. Riemann surface</h3>
</div>

<div style="background-color:  rgba(224, 224, 224, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(192, 192, 192); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(160, 160, 160);">Definition 1.2. </h4>
    <p>    
        A Riemann surface $X$ is a connected two-dimensional topological manifold with a complex-analytic structure on it. 
    </p>    
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
    The SageMath RiemannSurface library provides a function to generate the appropriate Riemann surface, on which we will continue our work.
</p>
</div>

In [6]:
S = RiemannSurface(f, prec=100)

<div>
<h3 style="font-family: 'Latin Modern Roman', Times, serif; font-size: 24px;">1.4. Branch points</h3>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
    We define a function to compute and display all branch points $e_i$ based on the given coefficients $\lambda_{2i}$. Branch points correspond to the zeros of the polynomial defined by $f(x, y) = 0$. Specifically, the polynomial is given by:
$$
    y^2 = (x - e_1)(x - e_2)(x - e_3)(x - e_4)(x - e_5).
$$
Here, the branch points $e_i$ are the roots of the polynomial on the right-hand side, which describe the structure of the Riemann surface associated with $f(x, y) = 0$.
</p>
</div>

In [7]:
def find_branch_points(ll2, ll4, ll6, ll8, ll10):
    # We define a ring of polynomials over the field of complex numbers
    CC_poly.<x> = PolynomialRing(CC)
    
    # We create a polynomial
    pol = x^5 + ll2*x^4 + ll4*x^3 + ll6*x^2 + ll8*x + ll10
    
    # We find roots
    roots = pol.roots(multiplicities=False)
    
    # We add a point at infinity if the degree of the polynomial is odd
    if pol.degree() % 2 == 1:
        roots.append(infinity)
    
    return roots

<div style="background-color:  rgba(153, 204, 255, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(0, 128,255); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(0, 128,255);">Note 1.3.</h4>
    <p>
        The <code style="color: rgb(81,206,9);font-size: 13px;">multiplicities=False</code> parameter in the <code style="color: rgb(81,206,9);font-size: 13px;">roots()</code>  method in Sage has the following meaning:
        <ul>
            <li>When <code style="color: rgb(81,206,9);font-size: 13px;">multiplicities=False</code>  (default): The method returns only the roots of the polynomial, without information about their multiplicities. The result is a list of unique root values.</li>
            <li>When <code style="color: rgb(81,206,9);font-size: 13px;">multiplicities=True</code>: The method returns pairs (root, multiplicity) for each root. The result is a list of tuples, where each tuple contains the root and its multiplicity.</li>
        </ul>            
    </p>
</div>



In [8]:
# Example of use:

branch_points = find_branch_points(l2, l4, l6, l8, l10)
print("Branch points:", *branch_points, sep='\n')

Branch points:
-11.8251161409669
-1.05264146874943
-0.512317960119317
0.695037784917798 - 1.04164546422316*I
0.695037784917798 + 1.04164546422316*I
+Infinity


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h1 style="font-size: 32px;">2. First and Second Kind Periods</h1>
<p>
<p>
    To construct periodic functions, we first define a canonical basis for the space of holomorphic differentials, \( \{du_i \mid i = 1, \ldots, g\} \), and the associated meromorphic differentials, \( \{dr_i \mid i = 1, \ldots, g\} \), on the Riemann surface as follows:
    $$
        du_{2i-1} := \frac{x^{g-i} dx}{\partial_y f(x,y)},
    $$
    $$
        dr_{2i-1} := \frac{\mathcal{R}_{2i-1}(x) dx}{\partial_y f(x,y)}, 
    $$
    where
    $$
    \mathcal{R}_{2i-1}(x)=\sum_{k=1}^{2i-1}k\lambda_{4i-2k-2}x^{g-i+k}.
    $$
    For \( g = 2 \), these expressions can be written in vector form as:
    $$
        du= \begin{pmatrix} 
                x\\
                1
            \end{pmatrix} \frac{dx}{-2\sqrt{f(x,y)}},
    $$
    $$
        dr= \begin{pmatrix} 
                \mathcal{R}_{1}(x)\\
                \mathcal{R}_{3}(x)
            \end{pmatrix} \frac{dx}{-2\sqrt{f(x,y)}} = 
                 \begin{pmatrix} 
                    x^2\\
                    3x^3 + 2\lambda_2 x^2 + \lambda_4 x
                \end{pmatrix} \frac{dx}{-2\sqrt{f(x,y)}},
    $$
</p>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
    The holomorphic basis defined above can be compared to the output of the <code style="color: rgb(81,206,9);font-size: 13px;">cohomology_basis()</code> function. In SageMath, <code style="color: rgb(81,206,9);font-size: 13px;">S.cohomology_basis()</code> generates a list of holomorphic differentials, typically represented as polynomials $g(x)$ corresponding to the differentials:
    $$
        \omega = g(x,y) \frac{\, dx}{\partial f / \partial y}
    $$
    where  $f(x, y) = 0$  defines the curve.
</p>
</div>    

In [9]:
S.cohomology_basis()

[1, x]

<div style="background-color:  rgba(153, 204, 255, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(0, 128,255); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(0, 128,255);">Note 2.1.</h4>
    <p>
    The order of elements clearly differs from the convention adopted in <a href="https://arxiv.org/abs/2407.05632" target="_blank">[1]</a>. In this notebook, we adopt Julia Bernatska’s convention, as it provides a consistent framework for our computations.
    </p>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h2 style="font-size: 24px;">2.1. First  Kind Periods</h2>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
   First we define our holomorphic base
</p>
</div>

In [10]:
# holomorphic differentials base
holbais=[x,x^0]

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
    To compute the period matrices of the first kind, one can evaluate the following integrals along the canonical homology cycles $\{\mathfrak{a}_i, \mathfrak{b}_i\}_{i=1}^g$:
    $$
        \omega = (\omega_{ij})= \left( \int_{\mathfrak{a}_j}du_i \right), \quad \omega' = (\omega'_{ij})= \left( \int_{\mathfrak{b}_j}du_i \right). 
    $$
Here, $\omega$ and $\omega'$ are the period matrices corresponding to the $\mathfrak{a}$ and $\mathfrak{b}$ cycles, respectively.

Instead of this we use the SageMath function <code style="color: rgb(81,206,9);font-size: 13px;">matrix_of_integral_values(differentials, integration_method='heuristic')</code> which compute the path integrals of the given differentials along the homology basis. The result is a matrix, where each row corresponds to a differential.

If the Riemann surface is given by the equation $f(x,y)=0$, the differentials are encoded by:
    $$
            g(x,y)\frac{dx}{(df/dy)}.
    $$
    <code style="color:  rgb(229,201,18);font-size: 13px;">Input:</code>
    <ul>
        <li><code style="color:  rgb(81,206,9);font-size: 13px;">differentials</code> – a list of polynomials.</li>
        <li><code style="color:  rgb(81,206,9);font-size: 13px;">integration_method</code> – (default: 'heuristic'). String specifying the integration method to use. The options are 'heuristic' and 'rigorous'.</li>
    </ul>
    <code style="color:  rgb(229,201,18);font-size: 13px;">Output:</code>
    A matrix, one row per differential, containing the values of the path integrals along the homology basis of the Riemann surface.
</p>
</div>

In [11]:
MofInt1=S.matrix_of_integral_values(holbais)
# Let's display the matrix in a shortened form so that it will be easy to see its structure
print(MofInt1.n(digits=5))

[     0.13760 - 0.66033*I  -4.9304e-32 - 0.91722*I  -3.9443e-31 + 0.40344*I       1.1746 + 0.25689*I]
[    -0.62448 + 0.28220*I -5.5467e-32 + 0.038862*I  -4.9304e-32 - 0.52554*I      0.14607 + 0.24334*I]


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
    The structure of the returned matrix
    $$
    MofInt1 = 
        \begin{pmatrix}
            \omega_{1,1} & \omega_{1,2} & \omega'_{1,1} & \omega'_{1,2} \\
            \omega_{3,1} & \omega_{3,2} & \omega'_{3,1} & \omega'_{3,2}
        \end{pmatrix}
        =
        \begin{array}{|c|c|c|c|c|}
            \hline
            & \mathfrak{a}_1 & \mathfrak{a}_2 & \mathfrak{b}_1 & \mathfrak{b}_2 \\
            \hline
            du_1 & \omega_{1,1} = \int_{\mathfrak{a}_1} du_1 & \omega_{1,2} = \int_{\mathfrak{a}_2} du_1 & \omega'_{1,1} = \int_{\mathfrak{b}_1} du_1 & \omega'_{1,2} = \int_{\mathfrak{b}_2} du_1 \\
            \hline
            du_3 & \omega_{3,1} = \int_{\mathfrak{a}_1} du_3 & \omega_{3,2} = \int_{\mathfrak{a}_2} du_3 & \omega'_{3,1} = \int_{\mathfrak{b}_1} du_3 & \omega'_{3,2} = \int_{\mathfrak{b}_2} du_3 \\
            \hline
        \end{array}
    $$
</p>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
    <p>
    We can compare this with the results of the built-in Sage function: <code style="color:  rgb(81,206,9);font-size: 13px;">period_matrix()</code>, which, for the adopted notational convention, will return a period matrix in the form
    $$
        pM= \begin{pmatrix}
                \omega_{3,1} & \omega_{3,2} & \omega'_{3,1} & \omega'_{3,2} \\
                \omega_{1,1} & \omega_{1,2} & \omega'_{1,1} & \omega'_{3,2}
        \end{pmatrix}  
    $$   
    </p>
</div>

In [12]:
pM=S.period_matrix()
print(pM.n(digits=5))

[   -0.62448 + 0.28220*I 4.3141e-32 + 0.038862*I -4.9304e-32 - 0.52554*I     0.14607 + 0.24334*I]
[    0.13760 - 0.66033*I  2.2187e-31 - 0.91722*I -5.9165e-31 + 0.40344*I      1.1746 + 0.25689*I]


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
    Before going any further, lat's define a function that will display the matrices in a approximated form so that we can compare them more easily.
</p>
</div>    

In [13]:
def format_complex(z, digits=5, threshold=1e-10):
    real = float(z.real())
    imag = float(z.imag())
    
    # We round very small values to zero
    if abs(real) < threshold:
        real = 0
        if abs(imag) < threshold:
            return "0"
        # We format the result  
        return f"{imag:.{digits}f}*I"
    
    if abs(imag) < threshold:
        # We format the result
        return f"{real:.{digits}f}"

    sign = "+" if imag > 0 else "-"
    return f"{real:.{digits}f} {sign} {abs(imag):.{digits}f}*I"


def ApproxM(matrix, digits=5, threshold=1e-10):
    rows, cols = matrix.nrows(), matrix.ncols()
    
    for i in range(rows):
        formatted_row = [format_complex(matrix[i,j], digits, threshold) for j in range(cols)]
        print("\t".join(formatted_row))

In [14]:
ApproxM(MofInt1)

0.13760 - 0.66033*I	-0.91722*I	0.40344*I	1.17459 + 0.25689*I
-0.62448 + 0.28220*I	0.03886*I	-0.52554*I	0.14607 + 0.24334*I


In [15]:
ApproxM(pM)

-0.62448 + 0.28220*I	0.03886*I	-0.52554*I	0.14607 + 0.24334*I
0.13760 - 0.66033*I	-0.91722*I	0.40344*I	1.17459 + 0.25689*I


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
    <p>
    In what follows we use the function <code style="color:  rgb(81,206,9);font-size: 13px;">matrix_of_integral_values()</code> instead of <code style="color:  rgb(81,206,9);font-size: 13px;">period_matrix()</code> because it allows us to calculate periodic matrices of the second kind.
    </p>
</div>

In [16]:
# Extract the omega-periods (first two columns)
omega = MofInt1[:, 0:2]

# Extract the omega'-periods (last two columns)
omegaP = MofInt1[:, 2:4]

ApproxM(omega)
print()
ApproxM(omegaP)

0.13760 - 0.66033*I	-0.91722*I
-0.62448 + 0.28220*I	0.03886*I

0.40344*I	1.17459 + 0.25689*I
-0.52554*I	0.14607 + 0.24334*I


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
    <p>
    Now we calculate the matrix
    $$
        \tau=\omega^{-1}\omega'
    $$
    which belongs to the Siegel upper half-space. Hence it should satisfy two conditions:
    <ul>
        <li>Symmetry: $$ \tau^T = \tau$$ 
        <li>Positive definiteness of the imaginary part $$ Im(\tau)>0$$
    </ul>
    </p>
</div>

In [17]:
tau= omega.inverse() * omegaP

# Displaying the result
print(tau.n(digits=5))

[-0.28894 + 0.70313*I -0.12636 - 0.46286*I]
[-0.12636 - 0.46286*I  -0.25854 + 1.6328*I]


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
    <p>
    Here we can also compare this with the results of the built-in Sage function: <code style="color:  rgb(81,206,9);font-size: 13px;">riemann_matrix()</code>:
    </p>
</div>

In [18]:
S.riemann_matrix().n(digits=5)

[-0.28894 + 0.70313*I -0.12636 - 0.46286*I]
[-0.12636 - 0.46286*I  -0.25854 + 1.6328*I]

In [19]:
# Test of the symmetry
print(tau-tau.transpose().n(digits=5))

[0.00000 0.00000]
[0.00000 0.00000]


In [20]:
# Test of positivity
# Calculating the complex part of the tau matrix
tauImag = tau.apply_map(lambda x: x.imag())

# Calculate the eigenvalues
eigenvalues = tauImag.eigenvalues()

# Checking if all eigenvalues are positive
all_positive = all(e > 0 for e in eigenvalues)

# Displaying the result
eigenvalues, all_positive

([1.8239307353738361091933820951, 0.51198440541283078719843599741], True)

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h2 style="font-size: 24px;">2.2. Second Kind Periods</h2>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
To calculate the second kind period matrices, we need to calculate the following integrals along the canonical homology cycles $\{ \mathfrak{a}_i, \mathfrak{b}_i\}_{i=1}^g$
    <div style="text-align: center;">
        $\eta = (\eta_{ij})= \left( \int_{\mathfrak{a}_j}dr_i \right), \quad \eta' = (\eta'_{ij})= \left( \int_{\mathfrak{b}_j}dr_i \right). $
    </div>
</p>
</div>

In [21]:
# meromorphic differentials base
merbais=[x^2, 3*x^3 + 2*l2*x^2 + l4*x]

In [22]:
MofInt2=S.matrix_of_integral_values(merbais)
# Let's display the matrix in a shortened form so that it will be easy to see its structure
ApproxM(MofInt2)

0.15574 + 0.20627*I	0.08372*I	-0.32882*I	-7.07532 + 0.12255*I
4.74867 + 3.09497*I	-0.04832*I	-6.23827*I	-2.96268 + 3.14329*I


In [23]:
# Extract the omega-periods (first two columns)
eta = MofInt2[:, 0:2]

# Extract the omega'-periods (last two columns)
etaP = MofInt2[:, 2:4]

ApproxM(eta)
print()
ApproxM(etaP)

0.15574 + 0.20627*I	0.08372*I
4.74867 + 3.09497*I	-0.04832*I

-0.32882*I	-7.07532 + 0.12255*I
-6.23827*I	-2.96268 + 3.14329*I


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
    <p>
    We can compute $\kappa$, given by
    $$
        \kappa=\eta\; \omega^{-1}
    $$
    </p>
</div>

In [24]:
omega_inv=Matrix(omega).inverse()
kappa = eta*omega_inv
ApproxM(kappa)

-0.09763 - 0.01261*I	-0.14978 - 0.29754*I
-0.14978 - 0.29754*I	-4.77835 - 7.02263*I


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h1 style="font-size: 24px;">3. Legendre relation</h1>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
    Next, we perform another test. The unnormalised period matrices of the first kind, $\omega$ and $\omega'$, and the second kind, $\eta$ and $\eta'$, should satisfy the Legendre relation:
    $$
        \Omega^T J \Omega = 2\pi i J
    $$
    where
    $$
        \Omega=
            \begin{pmatrix}
                \omega && \omega'\\
                \eta && \eta'\\
            \end{pmatrix}, \quad 
        J=        
            \begin{pmatrix}
                0 && -1_g\\
                1_g && 0\\
            \end{pmatrix}
    $$
</p>
</div>

In [25]:
# Omega matrix
Omega = block_matrix([
    [omega, omegaP],
    [eta, etaP]
])

# Converting lists to matrices
zeroM = Matrix([[0.0, 0.0], [0.0, 0.0]])
mOneg = Matrix([[-1.0, 0.0], [0.0, -1.0]])
Oneg = Matrix([[1.0, 0.0], [0.0, 1.0]])

# J matrix
J = block_matrix([
    [zeroM, mOneg],
    [Oneg, zeroM]
])    


print("Omega Matrix:")
print(Omega.n(digits=5))
print()
print("J Matrix:")
print(J.n(digits=5))

Omega Matrix:
[     0.13760 - 0.66033*I  -4.9304e-32 - 0.91722*I| -3.9443e-31 + 0.40344*I       1.1746 + 0.25689*I]
[    -0.62448 + 0.28220*I -5.5467e-32 + 0.038862*I| -4.9304e-32 - 0.52554*I      0.14607 + 0.24334*I]
[-------------------------------------------------+-------------------------------------------------]
[     0.15574 + 0.20627*I  7.8886e-31 + 0.083724*I| -8.3816e-31 - 0.32882*I      -7.0753 + 0.12255*I]
[       4.7487 + 3.0950*I  1.1360e-28 - 0.048319*I|   2.2088e-29 - 6.2383*I       -2.9627 + 3.1433*I]

J Matrix:
[0.00000 0.00000|-1.0000 0.00000]
[0.00000 0.00000|0.00000 -1.0000]
[---------------+---------------]
[ 1.0000 0.00000|0.00000 0.00000]
[0.00000  1.0000|0.00000 0.00000]


In [26]:
import numpy as np

pi = np.pi
left=Omega.transpose()*J*Omega
right = 2*pi*I*J
result = left - right
ApproxM(result)

0	0	0	0
0	0	0	0
0	0	0	0
0	0	0	0


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h1 style="font-size: 24px;">4. Theta function</h1>
<p>

</p>
</div>

<div style="background-color:  rgba(224, 224, 224, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(192, 192, 192); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(160, 160, 160);">Definition 4.1.</h4>
    <p>        
        The Riemann <i>theta function</i> $\theta(v;\tau)$ defined in terms of normalised coordinates $\mathbf{v} = \omega^{-1} \mathbf{u}$ and period matrix $\tau$, canonicaly is given by
        $$
            \theta(\mathbf{v};\tau) = \sum_{n\in\mathbb{Z}^g} e^{i\pi \mathbf{n}^T \tau \mathbf{n} + 2i \pi \mathbf{n}^T \mathbf{v}}.
        $$
        A theta function with characteristic $[\varepsilon ]$ is defined by
        $$
            \theta[\varepsilon](\mathbf{v};\tau) = 
            \theta 
                \begin{bmatrix}
                    \varepsilon'_1 & \ldots & \varepsilon'_g\\
                    \varepsilon_1 & \ldots & \varepsilon_g\\
                \end{bmatrix}
            (\mathbf{v};\tau)=
            \sum_{n\in\mathbb{Z}^g} e^{i\pi\{ (\mathbf{n}+ \frac{1}{2}\mathbf{\varepsilon}'^T ) \tau (\mathbf{n}+ \frac{1}{2}\mathbf{\varepsilon}') +2 (\mathbf{v}+\frac{1}{2}\mathbf{\varepsilon})^T (\mathbf{n}+\frac{1}{2}\mathbf{\varepsilon}') \} }.
        $$
    </p>    
    </div>

</div>

<div style="background-color:  rgba(255, 153, 153, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(255, 51,51); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(255, 51,51);">Important 4.1.</h4>
    <p>
        In the literature, it is common to use a convention with slightly different characteristics:
        $$
            (\varepsilon_i, \varepsilon'_j) \to 2 (\varepsilon_i, \varepsilon'_j)
        $$
        in consequence
        $$
            \theta[\varepsilon](\mathbf{v};\tau) = 
            \theta 
                \begin{bmatrix}
                    \varepsilon'_1 & \ldots & \varepsilon'_g\\
                    \varepsilon_1 & \ldots & \varepsilon_g\\
                \end{bmatrix}
            (\mathbf{v};\tau)=
            \sum_{n\in\mathbb{Z}^g} e^{i\pi\{ (\mathbf{n}+ \mathbf{\varepsilon}'^T ) \tau (\mathbf{n}+ \mathbf{\varepsilon}') +2 (\mathbf{v}+\mathbf{\varepsilon})^T (\mathbf{n}+\mathbf{\varepsilon}') \} }.
        $$
    </p>    
    </div>

</div>

In [27]:
def Theta( v, ttau, NAcc):
    # NAcc is responsible for the number of elements in the sum, i.e. the precision of the result. 
    # Experimentally, a good approximation is obtained for NAcc>4, but of course this can be increased as needed.
    total_sum = 0
    # epsilon_m is the list [epsilon 1, epsilon 2] where epsilon1 and epsilon2 are vectors
    
    # We iterate over two indices from -NAcc to NAcc
    for n1 in range(-NAcc, NAcc):
        for n2 in range(-NAcc, NAcc):
            # We create vector n
            n = vector([n1, n2])
                    
            # The first component of the sum
            term1 = I * pi * n * (ttau * n)
                    
            # The second component of the sum
            term2 = 2 * I * pi * n * v
                    
            # We add the exp from these components to the total
            total_sum += exp(term1 + term2)
    
    return total_sum

In [28]:
def ThetaCh(epsilon_m, v, ttau, NAcc):
    # NAcc is responsible for the number of elements in the sum, i.e. the precision of the result. 
    # Experimentally, a good approximation is obtained for NAcc>4, but of course this can be increased as needed.
    total_sum = 0
    # epsilon_m is the list [epsilon 1, epsilon 2] where epsilon1 and epsilon2 are vectors
    epsilon1 = epsilon_m[0]
    epsilon2 = epsilon_m[1]
    
    # We iterate over two indices from -NAcc to NAcc
    for n1 in range(-NAcc, NAcc):
        for n2 in range(-NAcc, NAcc):
            # We create vector n
            n = vector([n1, n2])
                    
            # The first component of the sum
            term1 = I * pi * (n + 1/2 * vector(epsilon1)) * (ttau * (n + 1/2 * vector(epsilon1)))
                    
            # The second component of the sum
            term2 = 2 * I * pi * (n + 1/2 * vector(epsilon1)) * (v + 1/2 * vector(epsilon2))
                    
            # We add the exp from these components to the total
            total_sum += exp(term1 + term2)
    
    return total_sum

<div style="background-color:  rgba(153, 204, 255, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(0, 128,255); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(0, 128,255);">Note 4.1.</h4>
    <p>
        In Wolfram Mathematica exists a corresponding function under the name <code style="color:  rgb(81,206,9);font-size: 13px;">SiegelTheta[$\nu_1$,$\nu_2$]($\Omega,s$)</code>. The relation between our variables and those in Mathematica are as follows
        <ul>
            <li> $\Omega=\tau$ </li>
            <li> $s=v$ </li>
            <li> $\nu_1 = \frac{1}{2} epsilon1$ </li>
            <li> $\nu_2 = \frac{1}{2} epsilon2$   </li> 
        </ul>  
        To test this, you can check the following code in Mathematica
    </p>
        <pre style=" color: #f8f8f2; padding: 1px; border-radius: 5px; font-size: 13px;">
        <code style="color: rgb(81,206,9);">
            ```Mathematica         
            SiegelTheta[{{2, 4}, {4, 2}}, IdentityMatrix[2] I, {1, 2}] // N
        </code>
        </pre>
    <p>
        and compare it with below one
    </p>    
        <pre style=" color: #f8f8f2; padding: 10px; border-radius: 5px; font-size: 13px;">
        <code style="color: rgb(81,206,9)">
            ```SageMath           
            # Defining sample data
            epsilon_m = [vector([4, 8]), vector([8, 4])]
            v = vector([1, 2])
            tau = Matrix([[I, 0], [0, I]])
            NAcc = 5
            # Function call
            result = ThetaCh(epsilon_m, v, tau, NAcc)
            print(result)
        </code>
        </pre>
    <p>
        Additionally, one can define the $\theta$ function in Mathematica in a similar way with the following code:
    </p>
        <pre style=" color: #f8f8f2; padding: 10px; border-radius: 5px; font-size: 13px;">
        <code style="color: rgb(81,206,9);">
        ```Mathematica
        ThetaCh[\[Epsilon]m_, v_, \[Tau]_, NAcc_] :=
         Sum[
          Exp[
           I Pi ({Subscript[n, 1], Subscript[n, 2]} + 
            1/2 \[Epsilon]m[[1]]) . (\[Tau] . ({Subscript[n, 1], Subscript[n, 2]} + 1/2 \[Epsilon]m[[1]])) + 2 I Pi ({Subscript[n, 1], Subscript[n, 2]} + 1/2 \[Epsilon]m[[1]]) . (v + 1/2 \[Epsilon]m[[2]])
           ],
          {Subscript[n, 2], -NAcc, NAcc}, {Subscript[n, 1], -NAcc, NAcc}
          ]
        </code>
        </pre>
    <p>
        And check the result of the above test with: 
    </p>
        <pre style=" color: #f8f8f2; padding: 10px; border-radius: 5px; font-size: 13px;">
        <code style="color: rgb(81,206,9);">
        ```Mathematica    
        ThetaCh[2 {{2, 4}, {4, 2}}, {1, 2}, IdentityMatrix[2] I, 5] // N
        </code>
        </pre>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h2 style="font-size: 24px;">5.1. Tests</h2>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
Test of the formula (<a href="https://arxiv.org/abs/2407.05632" target="_blank">[1]</a>, p.4, eq. 7)
$$
    \theta[\varepsilon](\mathbf{v};\tau) = 
            e^{i\pi\left(\frac{1}{2} \mathbf{\varepsilon}'^T\right) \tau \left(\frac{1}{2} \mathbf{\varepsilon}'\right) + 2i\pi \left( \mathbf{v} + \frac{1}{2} \mathbf{\varepsilon}\right)^T \left(\frac{1}{2} \mathbf{\varepsilon}'\right)} \theta\left(\mathbf{v} + \frac{1}{2}\mathbf{\varepsilon} + \tau\left(\frac{1}{2}\mathbf{\varepsilon}'\right);\tau \right).   
$$    

where a characteristic is a a $2\times g$ matrix $[\varepsilon] = (\mathbf{\varepsilon}',\mathbf{\varepsilon})^T $    
</p>
</div>

In [29]:
# Define the genus
g = 2  # For genus 2

Acc=20

# Define the period matrix tau
### tau = Matrix(CC, [[I, 0.5], [0.5, I]])
tau = omega.inverse() * omegaP

# Define the vector v
v = vector(CC, [0.1, 0.2])

# Define the characteristic
eps_prime = [1, 0]
eps = [0, 1]
N = 2  # Level of the characteristic
char = Matrix([eps_prime,eps])

# Compute theta with characteristic at v
theta_char = ThetaCh(char, v, tau, Acc)

# Compute half of the characteristic vectors
eps_vec = vector(CC, eps)
eps_prime_vec = vector(CC, eps_prime)
eps_half = 0.5 * eps_vec
eps_prime_half = 0.5 * eps_prime_vec

# Compute the shifted vector v_shifted
v_shifted = v + eps_half + tau * eps_prime_half

# Compute the terms for the exponential factor
term1 = (eps_prime_half) * tau * (eps_prime_half)
term2 = (v + eps_half) * (eps_prime_half)

# Compute the exponential factor
exp_factor = exp(I * pi * term1 + 2 * I * pi * term2)

# Compute theta at the shifted point without characteristic
theta_standard = Theta(v_shifted, tau, Acc)


# Compute the RHS of the relation
RHS = exp_factor * theta_standard

# Compute the difference
difference = theta_char - RHS

# Print the results
print("Theta with characteristic:\n", theta_char)
print()
print("Exponential factor:\n", exp_factor)
print()
print("Theta at shifted point:\n", theta_standard)
print()
print("RHS:\n", RHS)
print()
print("Difference:\n", difference)

# Check if the difference is within an acceptable tolerance
tolerance = 1e-12  # Adjust based on the precision
if abs(difference) < tolerance:
    print("The relation is verified within the given tolerance.")
else:
    print("The relation is not satisfied within the given tolerance.")

Theta with characteristic:
 1.0600858517074845 - 0.2501023731212625*I

Exponential factor:
 0.5734712442681018 + 0.050148801540618036*I

Theta at shifted point:
 1.7966652347912195 - 0.5932345951363659*I

RHS:
 1.0600858517074845 - 0.2501023731212626*I

Difference:
 1.1102230246251565e-16*I
The relation is verified within the given tolerance.


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h1 style="font-size: 24px;">5. $\sigma$-Functions</h1>
</div>

<div style="background-color:  rgba(224, 224, 224, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(192, 192, 192); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(160, 160, 160);">Definition 5.1.</h4>
    <p>        
        <em> Sigma function </em> (Kleinian sigma) is a modular invariant entire function on $\mathrm{Jac}(\mathscr{C})$. It is definef by a relation with the theta function:
        $$
            \sigma(\mathbf{u}) = C \tilde{\sigma}(\mathbf{u})
        $$
        where
        $$
            \tilde{\sigma}(\mathbf{u})= e^{-\frac{1}{2}\mathbf{u}^T \kappa \mathbf{u}} \theta[ K ] (\omega^{-1} \mathbf{u}, \tau),
        $$
        $$
            C= \sqrt{\frac{\pi^g}{\det{\omega}}} \left( \prod_{1\leq i<j \leq 2g+1} (e_i - e_j) \right)^{-1/4},
        $$
        and $[K]$ denotes the characteristics of the vector of Riemann constants. (The expression for $C$ comes from <a href="https://arxiv.org/pdf/1106.2408" target="_blank">[7]</a>, p.9, Eq. II.41)
    </p>    
    </div>

</div>

In [30]:
# We define variables
var('U1 U3')

# We define the accuracy of theta function
Acc=20


# sigma
def Tsigma(U1, U3):
    e = exp(-(1/2)*(vector([U1, U3])*kappa*vector([U1, U3])))
    theta = ThetaCh(KCh, omega_inv * vector([U1, U3]), tau, Acc)
    return e*theta

# C constant
det_omega = omega.determinant()
g = 2
#Branch points
BP = find_branch_points(l2, l4, l6, l8, l10)
# Calculating the product of branch point differences
prod = 1
for i in range(len(BP)):
    for j in range(i+1, len(BP)):
        if BP[i] != infinity and BP[j] != infinity:
            prod *= (BP[i] - BP[j])
C = sqrt(pi**g / det_omega) * prod**(-1/4)


def sigma(U1, U3):
    return C*Tsigma(U1, U3)

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
Let's define the derivatives of the sigma function which will be useful in the following sections.
</p>
</div>   

In [31]:
# sigma1
def Tsigma1(u1_val, u3_val):
    U1, U3 = var('U1 U3')
    sigma_expr = Tsigma(U1, U3)
    sigma1 =diff (sigma_expr, U1)
    return sigma1.subs({U1: u1_val, U3: u3_val}).n()

# sigma3
def Tsigma3(u1_val, u3_val):
    U1, U3 = var('U1 U3')
    sigma_expr = Tsigma(U1, U3)
    sigma3 =diff (sigma_expr, U3)
    return sigma3.subs({U1: u1_val, U3: u3_val}).n()

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h1 style="font-size: 24px;">5.2. $\sigma$-divisors</h1>
</div>

<div style="background-color:  rgba(224, 224, 224, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(192, 192, 192); padding: 15px; border-radius: 5px; ">
    <h3 style="font-size: 20px;color: rgb(160, 160, 160);">Definition 5.2.</h3>
    <p>        
        $\sigma$-divisors or <i>special divisors</i> are points $\mathbf{u}$ for which 
        $$
            \sigma(\mathbf{u})=0
        $$
    </p>    
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h1 style="font-size: 24px;">6. Characteristics of branch points and Bolza formula</h1>
</div>

<div style="background-color:  rgba(224, 224, 224, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(192, 192, 192); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(160, 160, 160);">Definition 6.1.</h4>
    <p>   
    A characteristic is a $2\times g$ matrix $[\varepsilon] = (\mathbf{\varepsilon}', \mathbf{\varepsilon})^T$ with real values within the interval $[0,2)$.
    Every point $\mathbf{u}$ in the funcamental domain $\mathrm{Jac}(\mathscr{C})$ can be represented by its characteristic $[\varepsilon]$:
        $$
             \mathbf{u}=\mathbf{u}[\varepsilon] = \frac{1}{2} \omega \mathbf{\varepsilon} + \frac{1}{2}\omega' \mathbf{\varepsilon}'.
        $$
    In the hyperelliptic case, the Abel images of branch points and any combination of branch points are described by characteristics with 1 or 0, which are called <i>half-integer</i> characteristics.
</p>
</div>   

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
    The half-integer characteristics are odd whenever 
    $$
        \varepsilon^T \varepsilon'=0 \pmod{2},
    $$
    and even when 
    $$
        \varepsilon^T \varepsilon'= 1\pmod{2},
    $$
    For hyperelliptic curves of genus 2, the characteristics are widely known in the literature <a href="https://arxiv.org/pdf/1106.2408" target="_blank">[7]</a>, <a href="https://www.itp.uni-bremen.de/prichter/download/ThetaConst.pdf" target="_blank">[12]</a>. To write them down, we can use the code proposed by Julia Bernatska <a href="https://community.wolfram.com/groups/-/m/t/3243472" target="_blank">[2]</a>
</p>
</div>

In [32]:
# We define the genus variable
genus = S.genus  


eChars = [[[0 for k in range(genus)], [1 for k in range(genus)]], 
          [[0 for k in range(genus)] for i in range(2)]]

# Adaptation of a Do loop from Julia Bernatska’s Mathematica notebook
for l in range(genus):
    # let's note the indexing, which must be adjusted by
    eChars.insert(0, 
        [[(eChars[0][0][k] + kronecker_delta(k+1, genus - l) + 
           kronecker_delta(k+1, genus - l + 1)) % 2 for k in range(genus)],
         [(eChars[0][1][k] + 0) % 2 for k in range(genus)]]
    )

    eChars.insert(0,
        [[(eChars[0][0][k] + 0) % 2 for k in range(genus)],
         [(eChars[0][1][k] + kronecker_delta(k+1, genus - l)) % 2 for k in range(genus)]]
    )

# We display matrices
seen_matrices = []
for i in range(len(eChars)):
    current_matrix = matrix(eChars[i])
    if current_matrix not in seen_matrices:
        seen_matrices.append(current_matrix)
        print(current_matrix)
        print()

[1 0]
[0 0]

[1 0]
[1 0]

[0 1]
[1 0]

[0 1]
[1 1]

[0 0]
[1 1]

[0 0]
[0 0]



<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
    There are 2 odd characteristic among them, their sum is also odd. The corresponding vector -vector of Riemann constants -is denoted by $\mathbf{K}$ and 
    $$
        [\mathbf{K}] = \sum_{\text{all odd}\; [\varepsilon_i]} [\varepsilon_i]
    $$
</p>
</div>

In [33]:
# We sum the eChars elements with indices 2*i +1 (because python counts from 0) and take Mod 2
KCh = sum(matrix(eChars[2 * i+1]) for i in range(genus)) % 2

print(KCh)

[1 1]
[0 1]


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
    The first test of the above functions and selected characteristics can be the Bolza formula given as follows
</p>
</div>

<div style="background-color:  rgba(224, 224, 224, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(192, 192, 192); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(160, 160, 160);">Definition 6.2.</h4>
    <p> From <a href="https://arxiv.org/abs/2407.05632" target="_blank">[1]</a>. (Sec. 2.4), we have the Bolza formula:
        $$
            e_\iota = - \frac{\partial_{u_3}\theta[{\iota}](\omega^{-1}\mathbf{u})}{\partial_{u_1}\theta[{\iota}](\omega^{-1}\mathbf{u})}|_{\mathbf{u}=0},
        $$
        where $[{\iota}] = [\varepsilon_\iota] +[K]$. It can also be defined in an equivalent way:
        $$
            e_\iota = - \frac{\sigma_3(\mathbf{u}_\iota)}{\sigma_1(\mathbf{u}_\iota)},
        $$
        where $\mathbf{u}_\iota=\mathbf{u}[\varepsilon_\iota]=\frac{1}{2}\omega \varepsilon_\iota + \frac{1}{2}\omega' \varepsilon'_\iota$, and $\sigma_i(\mathbf{u})$ denotes the derivative of the sigma function with respect to the $i$-th component of $\mathbf{u}$.
   <p>
   </div>   

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
   We will start by coding the second definition, which is easier to interpret.
</p>
</div>

In [34]:
# Characteristic vectors
epsp1 = vector(eChars[0][0])
eps1 = vector(eChars[0][1])

epsp2 = vector(eChars[1][0])
eps2 = vector(eChars[1][1])

epsp3 = vector(eChars[2][0])
eps3 = vector(eChars[2][1])

epsp4 = vector(eChars[3][0])
eps4 = vector(eChars[3][1])

epsp5 = vector(eChars[4][0])
eps5 = vector(eChars[4][1])

epsp6 = vector(eChars[5][0])
eps6 = vector(eChars[5][1])

# Points u
up1 = (1/2)*omega*eps1 + (1/2)*omegaP*epsp1
up2 = (1/2)*omega*eps2 + (1/2)*omegaP*epsp2
up3 = (1/2)*omega*eps3 + (1/2)*omegaP*epsp3
up4 = (1/2)*omega*eps4 + (1/2)*omegaP*epsp4
up5 = (1/2)*omega*eps5 + (1/2)*omegaP*epsp5
up6 = (1/2)*omega*eps6 + (1/2)*omegaP*epsp6

# ei based on sigma functions
def e(u1_val, u3_val):
    sigma1 = Tsigma1(u1_val, u3_val)
    sigma3 = Tsigma3(u1_val, u3_val)
    expr = -sigma3/sigma1
    return expr.n()

In [35]:
print("Branch points from Bolza formula:", 
      e(up1[0],up1[1]), 
      e(up2[0],up2[1]), 
      e(up3[0],up3[1]), 
      e(up4[0],up4[1]), 
      e(up5[0],up5[1]), 
      e(up6[0],up6[1]),sep='\n')

Branch points from Bolza formula:
-1.05264146874943 - 5.55111512312578e-16*I
0.695037784917798 + 1.04164546422316*I
-11.8251161409668 - 4.34097202628436e-14*I
2.53231492223134e15 - 6.84742393413673e15*I
0.695037784917799 - 1.04164546422316*I
-0.512317960119317


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
    We can see two things:
    <ol type="1">
        <li> the roots are unordered</li>
        <li> one of the roots has a very large magnitude.</li>
    </ol>
    Below, we present a simple program that reorders the characteristics listed above to arrange the roots $e_i$ in ascending order
    The fact that one of the roots has such a large magnitude (up to a sign precision) is a consequence of the fact that the point $\mathbf{u}$ is a <i>special divisor</i>, i.e. a point for which
    $$
        \sigma_1(\mathbf{u})=0
    $$
    The characteristic associated with this point will be the last one, corresponding to the root at infinity.
    
</p>
</div>

In [36]:
# List of characteristics and their corresponding brunch points
char = [eChars[0],eChars[1],eChars[2],eChars[3],eChars[4],eChars[5]]
values = [e(up1[0],up1[1]), e(up2[0],up2[1]), e(up3[0],up3[1]), e(up4[0],up4[1]), e(up5[0],up5[1]), e(up6[0],up6[1])]

# Bubble Sort Algorithm Implementation
def bubble_sort(char, values):
    sorted = []
    n = len(values)
    for i in range(n):
        for j in range(0, n-i-1):
            if values[j] > values[j+1]:  # Value comparison
                # Value replacing
                values[j], values[j+1] = values[j+1], values[j]
                # Replacing corresponding characteristics
                char[j], char[j+1] = char[j+1], char[j]
    
     # Making sure the infinite element is at the end
    if values[0].real().abs() > 10**10:
        values = values[1:] + values[:1]
        char = char[1:] + char[:1]
    
    return char, values    

In [37]:
# Sorting lists
char, values = bubble_sort(char, values)

In [38]:
# We display sorted characteristics matrices
seen_matrices = []
for i in range(len(char)):
    current_matrix = matrix(char[i])
    if current_matrix not in seen_matrices:
        seen_matrices.append(current_matrix)
        print(current_matrix)
        print()

[0 1]
[1 0]

[1 0]
[0 0]

[0 0]
[0 0]

[1 0]
[1 0]

[0 0]
[1 1]

[0 1]
[1 1]



In [39]:
print("Sorted values:", values)

Sorted values: [-11.8251161409668 - 4.34097202628436e-14*I, -1.05264146874943 - 5.55111512312578e-16*I, -0.512317960119317, 0.695037784917798 + 1.04164546422316*I, 0.695037784917799 - 1.04164546422316*I, 2.53231492223134e15 - 6.84742393413673e15*I]


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
    Now, we can redefine the characteristic vectors based on the reordered characteristics.
</p>
</div>    

In [40]:
# New characteristic vectors
epsp1 = vector(char[0][0])
eps1 = vector(char[0][1])

epsp2 = vector(char[1][0])
eps2 = vector(char[1][1])

epsp3 = vector(char[2][0])
eps3 = vector(char[2][1])

epsp4 = vector(char[3][0])
eps4 = vector(char[3][1])

epsp5 = vector(char[4][0])
eps5 = vector(char[4][1])

epsp6 = vector(char[5][0])
eps6 = vector(char[5][1])

# Points u
up1 = (1/2)*omega*eps1 + (1/2)*omegaP*epsp1
up2 = (1/2)*omega*eps2 + (1/2)*omegaP*epsp2
up3 = (1/2)*omega*eps3 + (1/2)*omegaP*epsp3
up4 = (1/2)*omega*eps4 + (1/2)*omegaP*epsp4
up5 = (1/2)*omega*eps5 + (1/2)*omegaP*epsp5
up6 = (1/2)*omega*eps6 + (1/2)*omegaP*epsp6

print("Branch points from Bolza formula:", 
      e(up1[0],up1[1]), 
      e(up2[0],up2[1]), 
      e(up3[0],up3[1]), 
      e(up4[0],up4[1]), 
      e(up5[0],up5[1]), 
      e(up6[0],up6[1]),sep='\n')

Branch points from Bolza formula:
-11.8251161409668 - 4.34097202628436e-14*I
-1.05264146874943 - 5.55111512312578e-16*I
-0.512317960119317
0.695037784917798 + 1.04164546422316*I
0.695037784917799 - 1.04164546422316*I
2.53231492223134e15 - 6.84742393413673e15*I


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
Before we go any further, we can do one more test. Based on the discussion <a href="https://community.wolfram.com/groups/-/m/t/3296279" target="_blank">[15]</a> in the Wolfram community, about the vector of Riemann constants for a computed characteristic $[\mathbf{K}]$, we should check whether
$$
        \sigma\left( \mathbf{u}[\mathbf{K}]\right) =0 \;\; \text{and} \;\; \sigma_1\left( \mathbf{u}[\mathbf{K}]\right) =0,
$$        
where
$$
        \mathbf{u}[\mathbf{K}]=
            \frac{1}{2}\omega 
                \begin{pmatrix}
                    K_{1,1}\\
                    K_{2,1}
                \end{pmatrix}  + 
            \frac{1}{2}\omega' 
                \begin{pmatrix}
                    K_{1,2}\\
                    K_{2,2}
                \end{pmatrix} 
        $$
</p>
</d>

In [41]:
vK1 = vector(KCh[0])
vK2 = vector(KCh[1])
uK = (1/2)*omega*vK1 + (1/2)*omegaP*vK2

print("sigma(u[K])=",Tsigma(uK[0], uK[1]))
print("sigma1(u[K])=",Tsigma1(uK[0], uK[1]))

sigma(u[K])= 4.168869563487747e-16 + 4.900022606845388e-16*I
sigma1(u[K])= 1.23432198410658e-15 + 3.52430396044750e-15*I


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
    We will now test the definition defined using theta functions, which will be referred to in the following parts of the code.
</p>
</div>

In [42]:
def e(i):
    # Declare variables
    U1, U3, = var('U1 U3')
    CCh=(matrix(char[i])+KCh)%2
    # Partial derivatives of the ThetaCh function
    numerator = diff(ThetaCh(CCh, omega_inv * vector([U1, U3]), tau, Acc), U3)
    denominator = diff(ThetaCh(CCh, omega_inv * vector([U1, U3]), tau, Acc), U1)

    # Simplified ratio of derivatives
    result = -numerator / denominator

    # Substitute U1, U3, -> 0
    result = result.subs({U1: 0, U3: 0})
    return result.n()


In [43]:
print("Branch points from Bolza formula:", 
      e(0),
      e(1), 
      e(2), 
      e(3), 
      e(4), 
      e(5), sep='\n')

Branch points from Bolza formula:
-11.8251161409669 + 1.46420013867900e-15*I
-1.05264146874943 - 3.37524010891116e-16*I
-0.512317960119317 + 3.11685236212985e-17*I
0.695037784917798 + 1.04164546422316*I
0.695037784917798 - 1.04164546422316*I
-2.69482572518105e17 - 2.55665399144444e17*I


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
    We can see that the last element has changed its value. This is, of course, a numerical error arising from computations involving very large numbers.
</p>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h1 style="font-size: 24px;">7. $\wp$-Functions</h1>
</div>

<div style="background-color:  rgba(224, 224, 224, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(192, 192, 192); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(160, 160, 160);">Definition 7.1.</h4>
    <p>
        Let $\Sigma = \{ \mathbf{u}\in \mathrm{Jac}(\mathscr{C})| \sigma(\mathbf{u})=0 \} $, then we define
        a multiply periodic Klein-Weierstrass $\wp$-functions on  $\mathrm{Jac}(\mathscr{C}) \setminus \Sigma$ by
        $$
            \wp_{ij}(\mathbf{u}):= - \frac{\partial^2\log{\sigma(\mathbf{u})}}{\partial u_i \partial u_j}, \quad \wp_{ijk}(\mathbf{u}):=- \frac{\partial^3\log{\sigma(\mathbf{u})}}{\partial u_i \partial u_j \partial u_k}.
        $$
        This can be also written by 
        $$
            \wp_{ij} (\mathbf{u})= \frac{\sigma_i(\mathbf{u})\sigma_j(\mathbf{u}) - \sigma(\mathbf{u})\sigma_{ij}(\mathbf{u})}{\sigma^2(\mathbf{u})} = \frac{\tilde{\sigma}_i(\mathbf{u})\tilde{\sigma}_j(\mathbf{u}) - \tilde{\sigma}(\mathbf{u})\tilde{\sigma}_{ij}(\mathbf{u})}{\tilde{\sigma}^2(\mathbf{u})},
        $$
        where $\sigma_i(\mathbf{u})$ denotes the derivative of the sigma function with respect to the $i$-th component of $\mathbf{u}$. <br>
        <br>
        It can be shown that the above definitions can be written in the form
        $$
            \wp_{ij}(\mathbf{u}):=\kappa_{ij} - \frac{\partial^2}{\partial u_i \partial u_j}\log{\theta[K](\omega^{-1}\mathbf{u};\tau)}, \quad \wp_{ijk}(\mathbf{u}):=- \frac{\partial^3}{\partial u_i \partial u_j \partial u_k}\log{\theta[K](\omega^{-1}\mathbf{u};\tau)}.
        $$
    </p>    
    </div>

</div>

In [44]:
# Definition with thetas
# We define variables
var('U1 U3')

# We define the accuracy of theta function
Acc=20


# WeierstrassP11
def WeierstrassP11(u1_val, u3_val):
    symbolic_expr = kappa[0, 0] - diff(log(ThetaCh(KCh, omega_inv * vector([U1, U3]), tau, Acc)), U1, 2)
    return symbolic_expr.subs({U1: u1_val, U3: u3_val}).n()

# WeierstrassP13
def WeierstrassP13(u1_val, u3_val):
    symbolic_expr = kappa[0, 1] - diff(log(ThetaCh(KCh, omega_inv * vector([U1, U3]), tau, Acc)),  U1, U3)
    return symbolic_expr.subs({U1: u1_val, U3: u3_val}).n()

# WeierstrassP33
def WeierstrassP33(u1_val, u3_val):
    symbolic_expr = kappa[1, 1] - diff(log(ThetaCh(KCh, omega_inv * vector([U1, U3]), tau, Acc)), U3, 2)
    return symbolic_expr.subs({U1: u1_val, U3: u3_val}).n()

#####
# Derivatives of these functions useful in subsequent sections
#####

# WeierstrassP111
def WeierstrassP111(u1_val, u3_val):
    symbolic_expr = - diff(log(ThetaCh(KCh, omega_inv * vector([U1, U3]), tau, Acc)), U1, 3)
    return symbolic_expr.subs({U1: u1_val, U3: u3_val}).n()

# WeierstrassP113
def WeierstrassP113(u1_val, u3_val):
    symbolic_expr = - diff(log(ThetaCh(KCh, omega_inv * vector([U1, U3]), tau, Acc)), U1, U1, U3)
    return symbolic_expr.subs({U1: u1_val, U3: u3_val}).n()

# WeierstrassP133
def WeierstrassP133(u1_val, u3_val):
    symbolic_expr = - diff(log(ThetaCh(KCh, omega_inv * vector([U1, U3]), tau, Acc)),  U1, U3, U3)
    return symbolic_expr.subs({U1: u1_val, U3: u3_val}).n()

# WeierstrassP1111
def WeierstrassP1111(u1_val, u3_val):
    symbolic_expr = - diff(log(ThetaCh(KCh, omega_inv * vector([U1, U3]), tau, Acc)), U1, 4)
    return symbolic_expr.subs({U1: u1_val, U3: u3_val}).n()

# WeierstrassP1113
def WeierstrassP1113(u1_val, u3_val):
    symbolic_expr = - diff(log(ThetaCh(KCh, omega_inv * vector([U1, U3]), tau, Acc)), U1, U1, U1, U3)
    return symbolic_expr.subs({U1: u1_val, U3: u3_val}).n()

# WeierstrassP3333
def WeierstrassP3333(u1_val, u3_val):
    symbolic_expr = - diff(log(ThetaCh(KCh, omega_inv * vector([U1, U3]), tau, Acc)), U3, 4)
    return symbolic_expr.subs({U1: u1_val, U3: u3_val}).n()


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h2 style="font-size: 24px;">7.1. Tests</h2>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h3 style="font-size: 20px;">7.1.1 Periodicity</h3>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
    As a first test of these functions, we can check if they satisfy the key property  
$$
    \wp_{ij}(\mathbf{u}+2\omega \mathbf{n} + 2\omega'\mathbf{n}') = \wp_{ij}(\mathbf{u}),
$$
where
$$
    \mathbf{n} = 
        \begin{pmatrix}
            n_1\\
            n_2
        \end{pmatrix}, \quad
    \mathbf{n}' = 
        \begin{pmatrix}
            n'_1\\
            n'_2
        \end{pmatrix} \in \mathbb{Z}^2.
$$  
</p>
</d>

In [45]:
u1 = 0.212131
u3 = -2.231

ntest = vector([1, 2])
nPtest = vector([-3, -5])

wn = omega*ntest
wPn= omegaP*nPtest

print("Theta based definitions")
print()
print("Test P11:")
print(WeierstrassP11(u1, u3) - WeierstrassP11(u1 + 2*wn[0] + 2*wPn[0], u3 + 2*wn[1] + 2*wPn[1]))
print()
print("Test P13:")
print(WeierstrassP13(u1, u3) - WeierstrassP13(u1 + 2*wn[0] + 2*wPn[0], u3 + 2*wn[1] + 2*wPn[1]))
print()
print("Test P33:")
print(WeierstrassP33(u1, u3) - WeierstrassP33(u1 + 2*wn[0] + 2*wPn[0], u3 + 2*wn[1] + 2*wPn[1]))

Theta based definitions

Test P11:
5456.64098472225 - 129.384240060038*I

Test P13:
1691.55457358245 - 1547.13199513522*I

Test P33:
108.402730941999 - 956.649120634020*I


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
Let's check this for another value of vector $\mathbf{u}$

</p>
</div>

In [46]:
u1 = 2
u3 = 3

ntest = vector([1, 2])
nPtest = vector([-3, -5])

wn = omega*ntest
wPn= omegaP*nPtest

print("Theta based definitions")
print()
print("Test P11:")
print(WeierstrassP11(u1, u3) - WeierstrassP11(u1 + 2*wn[0] + 2*wPn[0], u3 + 2*wn[1] + 2*wPn[1]))
print()
print("Test P13:")
print(WeierstrassP13(u1, u3) - WeierstrassP13(u1 + 2*wn[0] + 2*wPn[0], u3 + 2*wn[1] + 2*wPn[1]))
print()
print("Test P33:")
print(WeierstrassP33(u1, u3) - WeierstrassP33(u1 + 2*wn[0] + 2*wPn[0], u3 + 2*wn[1] + 2*wPn[1]))

Theta based definitions

Test P11:
9.82254277914762e-11 + 4.30588897870621e-12*I

Test P13:
7.81597009336110e-12 + 1.25623955682386e-11*I

Test P33:
1.39550593303284e-11 - 1.67688085639384e-11*I


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
Let's check wheater this is a special divisor
</p>
</div>

In [48]:
u1 = 0.212131
u3 = -2.231

print("sigma(u)=",Tsigma(u1, u3))
print("sigma1(u)=",Tsigma1(u1, u3))
print("sigma3(u)=",Tsigma3(u1, u3))

sigma(u)= 6.79221453833314e7 + 1.01961715076165e8*I
sigma1(u)= -3.36302007714019e8 - 5.04841672720404e8*I
sigma3(u)= -1.11762167588201e9 - 1.67772413895504e9*I


<div style="background-color:  rgba(216, 220, 97, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(205,210,44); padding: 15px; border-radius: 5px; ">
    <h3 style="font-size: 20px;color: rgb(205,210,44);">Observation 7.1.</h3>
    <p>
    Similar to the case with real brunch points, but this time the first set of coordinates is the one that causes errors.
</p>
</d>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h3 style="font-size: 20px;">7.1.2 KdV equation</h3>
</div>

<div style="background-color:  rgba(224, 224, 224, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(192, 192, 192); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(160, 160, 160);">Definition 7.2.</h4>
    <p>  
        
Multiply periodic Klein-Weierstrass $\wp$-functions should satisfy the KdV equation given by (<a href="https://arxiv.org/pdf/2312.10859" target="_blank">[11]</a>, Eq. 44 and <a href="https://arxiv.org/abs/2412.10284" target="_blank">[14]</a>, Eq. 2.20):

$$
       \wp_{1111}(\mathbf{u}) = 6\wp^2_{11}(\mathbf{u}) +  4 \wp_{13}(\mathbf{u}) + 4\lambda_2 \wp_{11}(\mathbf{u}) + 2 \lambda_4 
    $$

</pre>
</div>

In [49]:
u1 = 0.212131
u3 = -2.231

LHS = WeierstrassP1111(u1,u3)
RHS = 6 * WeierstrassP11(u1,u3)^2 + 4 * l2 * WeierstrassP11(u1,u3) + 4 * WeierstrassP13(u1,u3) + 2*l4

LHS - RHS

-1.59161572810262e-12 - 2.75743734288691e-12*I

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
Let's check this for another value of vector $\mathbf{u}$

</p>
</div>

In [50]:
u1 = -1.98213
u3 = 3.76213

LHS = WeierstrassP1111(u1,u3)
RHS = 6 * WeierstrassP11(u1,u3)^2 + 4 * l2 * WeierstrassP11(u1,u3) + 4 * WeierstrassP13(u1,u3) + 2*l4

LHS - RHS

9.63495949690696e-12 + 9.83391146291979e-12*I

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h3 style="font-size: 20px;">7.1.3 Basic relations</h3>
</div>

<div style="background-color:  rgba(224, 224, 224, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(192, 192, 192); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(160, 160, 160);">Definition 7.3.</h4>
    <p>  
        
Multiply periodic Klein-Weierstrass $\wp$-functions should satisfy following relations (<a href="https://arxiv.org/abs/2412.10284" target="_blank">[14]</a>, Eq. 2.20):
    $$
       \wp_{133}(\mathbf{u}) = \wp_{13}(\mathbf{u})\wp_{111}(\mathbf{u}) -  \wp_{11}(\mathbf{u})\wp_{113}(\mathbf{u})
    $$
    $$
       \wp_{1113}(\mathbf{u}) = 6 \wp_{13}(\mathbf{u}) \wp_{11}(\mathbf{u}) - 2\wp_{33}(\mathbf{u}) + 4\lambda_2 \wp_{13}(\mathbf{u})
    $$
    $$
       \wp_{33}(\mathbf{u}) = \frac{1}{4} \wp_{111}^2(\mathbf{u})- \wp_{11}^3(\mathbf{u}) - \wp_{13}(\mathbf{u}) \wp_{11}(\mathbf{u}) - \lambda_2 \wp_{11}^2(\mathbf{u}) - \lambda_4\wp_{11}(\mathbf{u}) - \lambda_6
    $$
</pre>
</div>

In [53]:
u1 = 0.212131
u3 = -2.231

LHS1 = WeierstrassP133(u1,u3)
RHS1 = WeierstrassP13(u1,u3) * WeierstrassP111(u1,u3) - WeierstrassP11(u1,u3)*WeierstrassP113(u1,u3)

LHS2 = WeierstrassP1113(u1,u3)
RHS2 = 6*WeierstrassP13(u1,u3) * WeierstrassP11(u1,u3) - 2*WeierstrassP33(u1,u3) + 4*l2*WeierstrassP13(u1,u3)

LHS3 = WeierstrassP33(u1,u3)
RHS3 = (1/4) * WeierstrassP111(u1,u3)^2 - WeierstrassP11(u1,u3)^3 - WeierstrassP13(u1,u3)*WeierstrassP11(u1,u3) - l2*WeierstrassP11(u1,u3)^2 - l4*WeierstrassP11(u1,u3) - l6

print("Test of the first eq.:")
print(LHS1-RHS1)
print()
print("Test of the second eq.:")
print(LHS2-RHS2)
print()
print("Test of the third eq.:")
print(LHS3-RHS3)

Test of the first eq.:
-2.67164068645798e-12 - 4.88393017313736e-12*I

Test of the second eq.:
-1.38129507831763e-11 - 2.24519098308128e-11*I

Test of the third eq.:
3.66995323020092e-12 - 3.55449029934026e-12*I


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
Let's check this for another value of vector $\mathbf{u}$

</p>
</div>

In [54]:
u1 = -1.98213
u3 = 3.76213

LHS1 = WeierstrassP133(u1,u3)
RHS1 = WeierstrassP13(u1,u3) * WeierstrassP111(u1,u3) - WeierstrassP11(u1,u3)*WeierstrassP113(u1,u3)

LHS2 = WeierstrassP1113(u1,u3)
RHS2 = 6*WeierstrassP13(u1,u3) * WeierstrassP11(u1,u3) - 2*WeierstrassP33(u1,u3) + 4*l2*WeierstrassP13(u1,u3)

LHS3 = WeierstrassP33(u1,u3)
RHS3 = (1/4) * WeierstrassP111(u1,u3)^2 - WeierstrassP11(u1,u3)^3 - WeierstrassP13(u1,u3)*WeierstrassP11(u1,u3) - l2*WeierstrassP11(u1,u3)^2 - l4*WeierstrassP11(u1,u3) - l6

print("Test of the first eq.:")
print(LHS1-RHS1)
print()
print("Test of the second eq.:")
print(LHS2-RHS2)
print()
print("Test of the third eq.:")
print(LHS3-RHS3)

Test of the first eq.:
-2.08046913030557e-11 + 3.70053756862678e-11*I

Test of the second eq.:
-9.37916411203332e-11 + 7.85378716560857e-11*I

Test of the third eq.:
1.17434950652751e-11 + 1.07062536925189e-11*I


<div style="background-color:  rgba(216, 220, 97, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(205,210,44); padding: 15px; border-radius: 5px; ">
    <h3 style="font-size: 20px;color: rgb(205,210,44);">Observation 7.3.</h3>
    <p>
    In this case there are no problems, but you can probably find points where they occur
</p>
</d>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h1 style="font-size: 24px;">8. Jacobi inversion problem </h1>     
</div>

<div style="background-color:  rgba(224, 224, 224, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(192, 192, 192); padding: 15px; border-radius: 5px; ">
    <h3 style="font-size: 20px;color: rgb(160, 160, 160);">Definition 8.1.</h3>
    <p>        
        Let 
        $$
            D=\sum_{i=1}^n P_i,
        $$
        be a divisor on a curve $\mathscr{C}$. We assume that $D$ is <b>non-special</b>, that is 
        <ul>
          <li> $ n\geq g$,
          <li> $D$ does not contain pairs of points connected by the hyperelliptic involution.
        </ul>
    </p>    
</div>

<div style="background-color:  rgba(224, 224, 224, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(192, 192, 192); padding: 15px; border-radius: 5px; ">
    <h3 style="font-size: 20px;color: rgb(160, 160, 160);">Definition 8.2.</h3>
    <p>        
        Let $du$ be a normalised first kind differentials, then an Abel map $\mathcal{A}$ can be constructed by:
        $$
            \mathcal{A}(P) = \int_{\infty}^P du, \quad P = (x,y) \in \mathscr{C}.
        $$
        Here infinity is used as the base point. The Abel image of divisor $D$ is computed by 
        $$
            \mathcal{A}(D) = \sum_{i=1}^n \mathcal{A}(P_i).
        $$
    </p>    
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
    Let $\mathbf{u}=\mathcal{A}(D)$ be the Abel image of a degree $g$ positive non-special divisor $D$ on the curve. Then $D$ is uniquely defined by the system of equations:
    $$
        \mathcal{R}_{2g}(x;\mathbf{u}) = x^g - \sum_{i=1}^g x^{g-1} \wp_{1,2i-1}(\mathbf{u}) =0,
    $$
    $$
        \mathcal{R}_{2g+1}(x,y;\mathbf{u}) = 2y + \sum_{i=1}^{g-i} x^{g-1} \wp_{1,1,2i-1}(\mathbf{u}) =0.
    $$
    So in case of $g=2$:    
    $$
        \mathcal{R}_4(x;\mathbf{u})=x^2 - x\wp_{11}(\mathbf{u})- \wp_{13}(\mathbf{u})=0,
    $$    
    $$
        \mathcal{R}_5 (x;\mathbf{u})= 2y + x\wp_{111}(\mathbf{u}) + \wp_{113}(\mathbf{u})=0.
    $$    
</p>
</d>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h2 style="font-size: 20px;">8.1. Jacobi inversion problem on branch points</h2>
    Based on the Riemann vanishing theorem and the inverse Jaccobi problem we can define the following properties.
</div>

<div style="background-color:  rgba(224, 224, 224, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(192, 192, 192); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(160, 160, 160);">Definition 8.3</h4>
    <p>  
        
Based on <a href="https://arxiv.org/pdf/1106.2408" target="_blank">[7]</a></li> (5.10-13) let's take
$$
    \Omega_{ij} = \frac{1}{2} \omega (\varepsilon_i + \varepsilon_j) +  \frac{1}{2} \omega' (\varepsilon'_i + \varepsilon'_j)+ \mathbf{K},
$$
then
$$
    \wp_{11}(\Omega_{ij}) = e_i + e_j, \quad \wp_{13}(\Omega_{ij}) = -e_i e_j,  \quad i,j=1,\ldots,5, \; i\neq j
$$
and
$$
    \wp_{33}(\Omega_{ij}) = e_i e_j (e_p + e_q + e_r) + e_p e_q e_r
$$
</pre>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
    Now we can test the above definition using $\Omega_{ij}$ calculated from characteristics $\varepsilon$, but also using the built-in Sage function <code style="color: #66d9ef;font-size: 13px;">abel_jacobi()</code>, 
   which expects lists of tuples in the format <code style="color: #66d9ef;font-size: 13px;font-size: 13px;">(v,P)</code>, where  <code style="color: #66d9ef;font-size: 13px;">v</code> is the multiplicity of the point in the divisor (in our case 1 for both points), and  <code style="color: #66d9ef;font-size: 13px;">P</code> is a tuple  <code style="color: #66d9ef;font-size: 13px;">(x,y)</code> representing the point on the curve. 

The multiplicity <code style="color: #66d9ef;font-size: 13px;">v</code> (also called valuation) in the context of divisors determines how many times a given point appears in the divisor. Here are some key points:
<ul>
    <li> For regular points on the curve that are neither singular points nor points at infinity, typically $v = 1$ or $v = -1$ is used.
$v = 1$ means the point appears positively in the divisor (it is "added").
    <li> $v = -1$ means the point appears negatively in the divisor (it is "subtracted").
    <li> If a point appears multiple times, $v$ can be greater than 1 or less than -1.
    <li> For special points (e.g., points at infinity or singular points), v may take other values depending on the local structure of the curve at that point.
</ul>

</p>
</div> 

<div style="background-color:  rgba(255, 153, 153, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(255, 51,51); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(255, 51,51);">Important 8.1.</h4>
    <p>
        <code style="color: #66d9ef;font-size: 13px;">abel_jacobi()</code> returns the $\mathbf{u}$ coordinates in the reverse order of the convention we adopted.
    </p>    
    </div>

</div>

In [59]:
## Definition of Omega_ij
vK1 = vector(KCh[0])
vK2 = vector(KCh[1])

Omega12 = (1/2)*omega*(eps1 + eps2 + vK1) + (1/2)*omegaP*(epsp1 + epsp2 + vK2)
Omega13 = (1/2)*omega*(eps1 + eps3 + vK1) + (1/2)*omegaP*(epsp1 + epsp3 + vK2)
Omega14 = (1/2)*omega*(eps1 + eps4 + vK1) + (1/2)*omegaP*(epsp1 + epsp4 + vK2) 
Omega15 = (1/2)*omega*(eps1 + eps5 + vK1) + (1/2)*omegaP*(epsp1 + epsp5 + vK2)
Omega23 = (1/2)*omega*(eps2 + eps3 + vK1) + (1/2)*omegaP*(epsp2 + epsp3 + vK2)
Omega24 = (1/2)*omega*(eps2 + eps4 + vK1) + (1/2)*omegaP*(epsp2 + epsp4 + vK2)
Omega25 = (1/2)*omega*(eps2 + eps5 + vK1) + (1/2)*omegaP*(epsp2 + epsp5 + vK2)
Omega34 = (1/2)*omega*(eps3 + eps4 + vK1) + (1/2)*omegaP*(epsp3 + epsp4 + vK2)
Omega35 = (1/2)*omega*(eps3 + eps5 + vK1) + (1/2)*omegaP*(epsp3 + epsp5 + vK2)
Omega45 = (1/2)*omega*(eps4 + eps5 + vK1) + (1/2)*omegaP*(epsp4 + epsp5 + vK2)

In [60]:
## Code for abel_jacobi()

# Branch points
e1 = e(0)
e2 = e(1)
e3 = e(2)
e4 = e(3)
e5 = e(4)

# Divisors

divisor12 = [(-1, (e1,0)),(1, (e2,0))]
divisor13 = [(-1, (e1,0)),(1, (e3,0))] 
divisor14 = [(-1, (e1,0)),(1, (e4,0))] 
divisor15 = [(-1, (e1,0)),(1, (e5,0))] 
divisor23 = [(-1, (e2,0)),(1, (e3,0))] 
divisor24 = [(-1, (e2,0)),(1, (e4,0))]
divisor25 = [(-1, (e2,0)),(1, (e5,0))] 
divisor34 = [(-1, (e3,0)),(1, (e4,0))]
divisor35 = [(-1, (e3,0)),(1, (e5,0))] 
divisor45 = [(-1, (e4,0)),(1, (e5,0))]

# Abels images
ai12 = S.abel_jacobi(divisor12)
ai13 = S.abel_jacobi(divisor13)
ai14 = S.abel_jacobi(divisor14)
ai15 = S.abel_jacobi(divisor15)
ai23 = S.abel_jacobi(divisor23)
ai24 = S.abel_jacobi(divisor24)
ai25 = S.abel_jacobi(divisor25)
ai34 = S.abel_jacobi(divisor34)
ai35 = S.abel_jacobi(divisor35)
ai45 = S.abel_jacobi(divisor45)

# Reversing the order of coordinates
AJ12 = (ai12[1],ai12[0])
AJ13 = (ai13[1],ai13[0])
AJ14 = (ai14[1],ai14[0])
AJ15 = (ai15[1],ai15[0])
AJ23 = (ai23[1],ai23[0])
AJ24 = (ai24[1],ai24[0])
AJ25 = (ai25[1],ai25[0])
AJ34 = (ai34[1],ai34[0])
AJ35 = (ai35[1],ai35[0])
AJ45 = (ai45[1],ai45[0])

################
# Adding the Riemann vector shift
################
vK1 = vector(KCh[0])
vK2 = vector(KCh[1])
uK = (1/2)*omega*vK1 + (1/2)*omegaP*vK2

AOmega12 = (AJ12[0]+uK[0], AJ12[1]+uK[1])
AOmega13 = (AJ13[0]+uK[0], AJ13[1]+uK[1])
AOmega14 = (AJ14[0]+uK[0], AJ14[1]+uK[1])
AOmega15 = (AJ15[0]+uK[0], AJ15[1]+uK[1])
AOmega23 = (AJ23[0]+uK[0], AJ23[1]+uK[1])
AOmega24 = (AJ24[0]+uK[0], AJ24[1]+uK[1])
AOmega25 = (AJ25[0]+uK[0], AJ25[1]+uK[1])
AOmega34 = (AJ34[0]+uK[0], AJ34[1]+uK[1])
AOmega35 = (AJ35[0]+uK[0], AJ35[1]+uK[1])
AOmega45 = (AJ45[0]+uK[0], AJ45[1]+uK[1])

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
    Compare the angles calculated in both ways
</ul>

</p>
</div> 

In [61]:
print("Omega12 - AOmega12=", (Omega12[0].n(digits=5) - AOmega12[0].n(digits=5),Omega12[1].n(digits=5) - AOmega12[1].n(digits=5)))
print("Omega13 - AOmega13=", (Omega13[0].n(digits=5) - AOmega13[0].n(digits=5),Omega13[1].n(digits=5) - AOmega13[1].n(digits=5)))
print("Omega14 - AOmega14=", (Omega14[0].n(digits=5) - AOmega14[0].n(digits=5),Omega14[1].n(digits=5) - AOmega14[1].n(digits=5)))
print("Omega15 - AOmega15=", (Omega15[0].n(digits=5) - AOmega15[0].n(digits=5),Omega15[1].n(digits=5) - AOmega15[1].n(digits=5)))
print("Omega23 - AOmega23=", (Omega23[0].n(digits=5) - AOmega23[0].n(digits=5),Omega23[1].n(digits=5) - AOmega23[1].n(digits=5)))
print("Omega24 - AOmega24=", (Omega24[0].n(digits=5) - AOmega24[0].n(digits=5),Omega24[1].n(digits=5) - AOmega24[1].n(digits=5)))
print("Omega25 - AOmega25=", (Omega25[0].n(digits=5) - AOmega25[0].n(digits=5),Omega25[1].n(digits=5) - AOmega25[1].n(digits=5)))
print("Omega34 - AOmega34=", (Omega34[0].n(digits=5) - AOmega34[0].n(digits=5),Omega34[1].n(digits=5) - AOmega34[1].n(digits=5))) 
print("Omega35 - AOmega35=", (Omega35[0].n(digits=5) - AOmega35[0].n(digits=5),Omega35[1].n(digits=5) - AOmega35[1].n(digits=5)))
print("Omega45 - AOmega45=", (Omega45[0].n(digits=5) - AOmega45[0].n(digits=5),Omega45[1].n(digits=5) - AOmega45[1].n(digits=5))) 

Omega12 - AOmega12= (1.3122, -0.47840)
Omega13 - AOmega13= (1.1399 - 0.66033*I, 0.14607 + 0.28220*I)
Omega14 - AOmega14= (1.3122 - 0.91722*I, -0.47840 + 0.038862*I)
Omega15 - AOmega15= (1.3122 - 1.3207*I, -0.47840 + 0.56440*I)
Omega23 - AOmega23= (-0.17232 - 0.25689*I, 0.62448 - 0.24334*I)
Omega24 - AOmega24= (-0.51377*I, -0.48668*I)
Omega25 - AOmega25= (-0.91722*I, 0.038862*I)
Omega34 - AOmega34= (0.17232 - 0.25689*I, -0.62448 - 0.24334*I)
Omega35 - AOmega35= (0.17232 - 0.66033*I, -0.62448 + 0.28220*I)
Omega45 - AOmega45= (0.13760 - 0.66033*I, -0.62448 + 0.28220*I)


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
    Let's check whether, despite these differences, we get the correct results for the inverse Jacobi problem.
</ul>

</p>
</div> 

In [62]:
print("P11(Omega12)=",WeierstrassP11(Omega12[0],Omega12[1]))
print("P11(AOmega12)=",WeierstrassP11(AOmega12[0],AOmega12[1]))
print("From theory: e1+e2=",e1+ e2)

P11(Omega12)= -12.8777576097163 - 4.44089209850063e-15*I
P11(AOmega12)= -12.8777576097163 + 6.52256026967279e-16*I
From theory: e1+e2= -12.8777576097163 + 1.12667612778789e-15*I


In [63]:
print("P13(Omega12)=",WeierstrassP13(Omega12[0],Omega12[1]))
print("P13(AOmega12)=",WeierstrassP13(AOmega12[0],AOmega12[1]))
print("From theory: -e1*e2=",-e1* e2)

P13(Omega12)= -12.4476076227599 + 1.77635683940025e-15*I
P13(AOmega12)= -12.4476076227599 - 4.10782519111308e-15*I
From theory: -e1*e2= -12.4476076227599 - 2.44998284463022e-15*I


In [64]:
print("P33(Omega12)=",WeierstrassP33(Omega12[0],Omega12[1]))
print("P33(AOmega12)=",WeierstrassP33(AOmega12[0],AOmega12[1]))
print("From theory: e1*e2(e3+e4+e5) + e3*e4*e5=",e1*e2*(e3+e4+e5) + e3*e4*e5)

P33(Omega12)= 10.1226150881406 - 1.77635683940025e-14*I
P33(AOmega12)= 10.1226150881406 + 1.77635683940025e-14*I
From theory: e1*e2(e3+e4+e5) + e3*e4*e5= 10.1226150881406 + 4.96992635363731e-15*I


In [65]:
print("P11(Omega13)=",WeierstrassP11(Omega13[0],Omega13[1]))
print("P11(AOmega13)=",WeierstrassP11(AOmega13[0],AOmega13[1]))
print("From theory: e1+e3=",e1+ e3)

P11(Omega13)= -12.3374341010861 - 4.44089209850063e-15*I
P11(AOmega13)= -12.1578646219938 + 4.08842072453786e-8*I
From theory: e1+e3= -12.3374341010862 + 1.49536866230030e-15*I


In [66]:
print("P11(Omega14)=",WeierstrassP11(Omega14[0],Omega14[1]))
print("P11(AOmega14)=",WeierstrassP11(AOmega14[0],AOmega14[1]))
print("From theory: e1+e4=", e1+ e4)

P11(Omega14)= -11.1300783560491 + 1.04164546422316*I
P11(AOmega14)= -11.1300783560491 + 1.04164546422316*I
From theory: e1+e4= -11.1300783560491 + 1.04164546422316*I


In [67]:
print("P11(Omega15)=",WeierstrassP11(Omega15[0],Omega15[1]))
print("P11(AOmega15)=",WeierstrassP11(AOmega15[0],AOmega15[1]))
print("From theory: e1+e5=", e1+ e5)

P11(Omega15)= -11.1300783560490 - 1.04164546422317*I
P11(AOmega15)= -11.1300783560491 - 1.04164546422316*I
From theory: e1+e5= -11.1300783560491 - 1.04164546422316*I


In [68]:
print("P11(Omega23)=",WeierstrassP11(Omega23[0],Omega23[1]))
print("P11(AOmega23)=",WeierstrassP11(AOmega23[0],AOmega23[1]))
print("From theory: e2+e3=", e2+ e3)

P11(Omega23)= -1.56495942886875 - 3.88578058618805e-16*I
P11(AOmega23)= -1.60036675615352 + 6.99244995239212e-9*I
From theory: e2+e3= -1.56495942886875 - 3.06355487269817e-16*I


In [69]:
print("P11(Omega24)=",WeierstrassP11(Omega24[0],Omega24[1]))
print("P11(AOmega24)=",WeierstrassP11(AOmega24[0],AOmega24[1]))
print("From theory: e2+e4=",e2+ e4)

P11(Omega24)= -0.357603683831629 + 1.04164546422316*I
P11(AOmega24)= -0.357603683831641 + 1.04164546422316*I
From theory: e2+e4= -0.357603683831631 + 1.04164546422316*I


In [70]:
print("P11(Omega25)=",WeierstrassP11(Omega25[0],Omega25[1]))
print("P11(AOmega25)=",WeierstrassP11(AOmega25[0],AOmega25[1]))
print("From theory: e2+e5=",e2+ e5)

P11(Omega25)= -0.357603683831631 - 1.04164546422316*I
P11(AOmega25)= -0.357603683831631 - 1.04164546422316*I
From theory: e2+e5= -0.357603683831632 - 1.04164546422316*I


In [71]:
print("P11(Omega34)=",WeierstrassP11(Omega34[0],Omega34[1]))
print("P11(AOmega34)=",WeierstrassP11(AOmega34[0],AOmega34[1]))
print("From theory: e3+e4=",e3+ e4)

P11(Omega34)= 0.182719824798481 + 1.04164546422316*I
P11(AOmega34)= 0.187452462896973 + 1.07462830934531*I
From theory: e3+e4= 0.182719824798481 + 1.04164546422316*I


In [72]:
print("P11(Omega35)=",WeierstrassP11(Omega35[0],Omega35[1]))
print("P11(AOmega35)=",WeierstrassP11(AOmega35[0],AOmega35[1]))
print("From theory: e3+e5=",e3+ e5)

P11(Omega35)= 0.182719824798477 - 1.04164546422316*I
P11(AOmega35)= 0.187452457028380 - 1.07462830675917*I
From theory: e3+e5= 0.182719824798481 - 1.04164546422316*I


In [73]:
print("P11(Omega45)=",WeierstrassP11(Omega45[0],Omega45[1]))
print("P11(AOmega45)=",WeierstrassP11(AOmega45[0],AOmega45[1]))
print("From theory: e4+e5=",e4+ e5)

P11(Omega45)= 1.39007556983559 - 6.71684929898220e-15*I
P11(AOmega45)= 1.39007556983560 + 9.43689570931383e-16*I
From theory: e4+e5= 1.39007556983560 + 2.22044604925031e-16*I


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h2 style="font-size: 24px;">8.2. General case</h2>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
As a point $P_i$ we choose
$$
    P_i = (x_i,y_i) = (x_i, y_{+}(x_i)),
$$
where $y_{+}(x) = + \sqrt{y^2(x)}$ (there is second option $y_{-}(x) = - \sqrt{y^2(x)}$ ). 
</p> 
</div>

In [74]:
# Definition of polynomial
def Poly(x):
    return x^5 + l2*x^4 + l4*x^3 + l6*x^2 + l8*x + l10

# Definition of y
def y(x):
    # Polynomial module
    modulus = abs(Poly(x))
    
    # Polynomial argument
    angle = arg(Poly(x))  
    
    # Correction of angle according to conditions
    corrected_angle = angle / 2 + pi if angle < 0 else angle / 2
    
    # Assembly of module and phase
    root = sqrt(modulus) * exp(I * corrected_angle)
    
    return root

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
    From the theory 
    $$
        \wp_{11}(\mathbf{u}) = x_1 + x_2, \quad \wp_{13}(\mathbf{u}) = -x_1 x_2,  
    $$
        and for 
    $$
        F(x,z) = \left[2\lambda_{10}  + \lambda_8 (x+z)\right] + xz\left[ 2\lambda_6 + \lambda_4(x+z) \right]  + x^2 z^2 \left[ 2\lambda_2 + (x+z) \right]
    $$
    $$
        \wp_{33}(\mathbf{u}) = \frac{F(x_1,x_2) - 2y_1 y_2}{4(x_1 - x_2)^2}
    $$
</p> 
</div>

In [75]:
def fun(x,z):
    return 2*l10 + l8*(x+z) + x*z*(2*l6 + l4*(x+z)) + x^2 * z^2 *(2*l2+(x+z))

def p33(x1,x2):
    numerator = fun(x1,x2) - 2*y(x1)*y(x2)
    denominator = 4*(x1-x2)^2
    return numerator/denominator

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
Let's recall brunch points
</p>
</div>

In [76]:
print("Branch points:", *branch_points, sep='\n')

Branch points:
-11.8251161409669
-1.05264146874943
-0.512317960119317
0.695037784917798 - 1.04164546422316*I
0.695037784917798 + 1.04164546422316*I
+Infinity


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
We compare the values of $\mathbf{u}$ calculated from <code style="color: #66d9ef;font-size: 13px;">abel_jacobi()</code> with the values from the theory for arbitrary points $x_1$ and $x_2$
</p> 
</div>

In [77]:
x1 = 6.5
x2 = 5

y1 = y(x1)
y2 = y(x2)


# Divisors
divisor = [(-1, (Infinity,0)),(1, (e5,  0)),(-1, (x1, y1)),(1, (x2, y2))]

# Abel map
aj = S.abel_jacobi(divisor)
# Reversing the order of coordinates
AJ = (aj[1],aj[0])

u1=AJ[0]
u3=AJ[1]

print("P11(u):=",WeierstrassP11(u1, u3))
print("From theory: x1+x2=",x1+ x2)
print()
print("P13(u)=",WeierstrassP13(u1, u3))
print("From theory: -x1*x2=",-x1*x2)
print()     
print("P33(u)=",WeierstrassP33(u1, u3))
print("From theory:",p33(x1,x2) )

P11(u):= 0.184891289907970 - 1.07467781012325*I
From theory: x1+x2= 11.5000000000000

P13(u)= 0.353993474282754 - 0.557285599518726*I
From theory: -x1*x2= -32.5000000000000

P33(u)= 12.4404821020362 + 6.09804458536710*I
From theory: -204.146616551396*sqrt(881/2) + 4284.93055555556


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
Let's change the points $x_1$ and $x_2$
</div>
</p>

In [78]:
x1 = 1.5
x2 = -0.5

y1 = y(x1)
y2 = y(x2)


# Divisors
divisor = [(-1, (Infinity,0)),(1, (e5,  0)),(-1, (x1, y1)),(1, (x2, y2))]

# Abel map
aj = S.abel_jacobi(divisor)
# Reversing the order of coordinates
AJ = (aj[1],aj[0])

u1=AJ[0]
u3=AJ[1]

print("P11(u):=",WeierstrassP11(u1, u3))
print("From theory: x1+x2=",x1+ x2)
print()
print("P13(u)=",WeierstrassP13(u1, u3))
print("From theory: -x1*x2=",-x1*x2)
print()     
print("P33(u)=",WeierstrassP33(u1, u3))
print("From theory:",p33(x1,x2) )

P11(u):= 1.77557997339084 - 1.22425273828305*I
From theory: x1+x2= 1.00000000000000

P13(u)= -0.552540873788430 + 1.23859942939941*I
From theory: -x1*x2= 0.750000000000000

P33(u)= -59.6555863818557 + 27.4989143355881*I
From theory: 2.66403194341274


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
Let's change the brunch point in the divisor definition on the first one which is real
</div>
</p>

In [82]:
x1 = 1.5
x2 = -0.5

y1 = y(x1)
y2 = y(x2)


# Divisors
divisor = [(-1, (Infinity,0)),(1, (e3,  0)),(-1, (x1, y1)),(1, (x2, y2))]

# Abel map
aj = S.abel_jacobi(divisor)
# Reversing the order of coordinates
AJ = (aj[1],aj[0])

u1=AJ[0]
u3=AJ[1]

print("P11(u):=",WeierstrassP11(u1, u3))
print("From theory: x1+x2=",x1+ x2)
print()
print("P13(u)=",WeierstrassP13(u1, u3))
print("From theory: -x1*x2=",-x1*x2)
print()     
print("P33(u)=",WeierstrassP33(u1, u3))
print("From theory:",p33(x1,x2) )

P11(u):= 0.999818095342272 - 9.42324884789514e-10*I
From theory: x1+x2= 1.00000000000000

P13(u)= 0.749923179704208 + 1.16553522389040e-9*I
From theory: -x1*x2= 0.750000000000000

P33(u)= 15.4500625172065 - 8.65000444605357e-8*I
From theory: 2.66403194341274


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
Ok, let's change the points $x_1$ and $x_2$ for it
</div>
</p>

In [83]:
x1 = 10.5
x2 = -8.5

y1 = y(x1)
y2 = y(x2)


# Divisors
divisor = [(-1, (Infinity,0)),(1, (e3,  0)),(-1, (x1, y1)),(1, (x2, y2))]

# Abel map
aj = S.abel_jacobi(divisor)
# Reversing the order of coordinates
AJ = (aj[1],aj[0])

u1=AJ[0]
u3=AJ[1]

print("P11(u):=",WeierstrassP11(u1, u3))
print("From theory: x1+x2=",x1+ x2)
print()
print("P13(u)=",WeierstrassP13(u1, u3))
print("From theory: -x1*x2=",-x1*x2)
print()     
print("P33(u)=",WeierstrassP33(u1, u3))
print("From theory:",p33(x1,x2) )

P11(u):= 4.73719784630354 + 4.00053699792235e-7*I
From theory: x1+x2= 2.00000000000000

P13(u)= 103.024851216997 + 6.27970203170847e-6*I
From theory: -x1*x2= 89.2500000000000

P33(u)= 1167.35969687998 + 0.0000704657058179237*I
From theory: 48.3847049517949


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
Let's try the remaining brunch points in the dividers definition
</div>
</p>

In [86]:
x1 = 10.5
x2 = -8.5

y1 = y(x1)
y2 = y(x2)


# Divisors
divisor = [(-1, (Infinity,0)),(1, (e2,  0)),(-1, (x1, y1)),(1, (x2, y2))]

# Abel map
aj = S.abel_jacobi(divisor)
# Reversing the order of coordinates
AJ = (aj[1],aj[0])

u1=AJ[0]
u3=AJ[1]

print("P11(u):=",WeierstrassP11(u1, u3))
print("From theory: x1+x2=",x1+ x2)
print()
print("P13(u)=",WeierstrassP13(u1, u3))
print("From theory: -x1*x2=",-x1*x2)
print()     
print("P33(u)=",WeierstrassP33(u1, u3))
print("From theory:",p33(x1,x2) )

P11(u):= 1.63612121389863 - 2.08993484740461e-8*I
From theory: x1+x2= 2.00000000000000

P13(u)= -1.85649109962442 + 4.48041603817728e-9*I
From theory: -x1*x2= 89.2500000000000

P33(u)= -26.4254878930462 + 1.54122091089448e-7*I
From theory: 48.3847049517949


In [87]:
x1 = 10.5
x2 = -8.5

y1 = y(x1)
y2 = y(x2)


# Divisors
divisor = [(-1, (Infinity,0)),(1, (e1,  0)),(-1, (x1, y1)),(1, (x2, y2))]

# Abel map
aj = S.abel_jacobi(divisor)
# Reversing the order of coordinates
AJ = (aj[1],aj[0])

u1=AJ[0]
u3=AJ[1]

print("P11(u):=",WeierstrassP11(u1, u3))
print("From theory: x1+x2=",x1+ x2)
print()
print("P13(u)=",WeierstrassP13(u1, u3))
print("From theory: -x1*x2=",-x1*x2)
print()     
print("P33(u)=",WeierstrassP33(u1, u3))
print("From theory:",p33(x1,x2) )

P11(u):= 83.3574365361692 - 6.51322716094249e-6*I
From theory: x1+x2= 2.00000000000000

P13(u)= 42.2292611986710 - 3.25483494734158e-6*I
From theory: -x1*x2= 89.2500000000000

P33(u)= 33.3170685572608 - 1.97860730399846e-6*I
From theory: 48.3847049517949


<div style="background-color:  rgba(216, 220, 97, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(205,210,44); padding: 15px; border-radius: 5px; ">
    <h3 style="font-size: 20px;color: rgb(205,210,44);">Observation 8.1.</h3>
    <p>
    As for now, there are no clear rules for selecting divisors.
</p>
</d>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h1 style="font-size: 32px;"> Literature </h1>
    <ol type="1">
        <li> 
            Julia Bernatska, <em>Computation of $\wp$-functions on plane algebraic curves</em>,  arXiv (2024),<br>
            URL: <a href="https://arxiv.org/abs/2407.05632" target="_blank">https://arxiv.org/abs/2407.05632</a> </li>
        <li>
            Julia Bernatska, <em>Uniformization of a genus 4 hyperelliptic curve with arbitrary complex branch points</em>, Wolfram Community (2024),<br>
            URL: <a href="https://community.wolfram.com/groups/-/m/t/3243472" target="_blank">https://community.wolfram.com/groups/-/m/t/3243472</a> </li>
        <li> 
            SageMath Documentation, <em>Riemann matrices and endomorphism rings of algebraic Riemann surfaces</em>,<br>
            URL:<a href="https://doc.sagemath.org/html/en/reference/curves/sage/schemes/riemann_surfaces/riemann_surface.html#sage.schemes.riemann_surfaces.riemann_surface.RiemannSurface"
target="_blank">https://doc.sagemath.org/html/en/reference/curves/sage/schemes/riemann_surfaces/riemann_surface.html#sage.schemes.riemann_surfaces.riemann_surface.RiemannSurface</a></li>
        <li> 
            V.M.Buchstaber, V.Z. Enolski, and D.V.Leykin, <em>Hyperelliptic Kleinian function and applications</em>, arXiv (1996), <br>
            URL: <a href="https://arxiv.org/pdf/solv-int/9603005" target="_blank">https://arxiv.org/pdf/solv-int/9603005</a></li>
        <li> 
            V.M.Buchstaber, V.Z. Enolski, and D.V.Leykin, <em>Multi-variable sigma-functions: old and new results</em>, arXiv (2018), <br>
            URL: <a href="https://arxiv.org/pdf/1810.11079" target="_blank">https://arxiv.org/pdf/1810.11079</a></li>
        <li> 
            J. C. Eilbeck, J. Gibbons, Y. Onishi, and S. Yasuda, <em>Theory of Heat Equations for Sigma Functions</em>, arXiv (2018), <br>
            URL: <a href="https://arxiv.org/pdf/1711.08395" target="_blank">https://arxiv.org/pdf/1711.08395</a></li> 
        <li> 
            V. Z. Enolski, B. Hartmann, V. Kagramanova, J. Kunz, C. Lammerzahl, P. Sirimachan, <em>Inversion of a general hyperelliptic integral and particle motion in Horava-Lifshitz black hole space-times </em>, arXiv (2011), <br>
            URL: <a href="https://arxiv.org/pdf/1106.2408" target="_blank">https://arxiv.org/pdf/1106.2408</a></li>
        <li>
            Athorne C., Eilbeck J.C., Enolskii V.Z., <em>A SL(2) covariant theory of genus 2 hyperelliptic functions</em>, Math. Proc. Cambridge Philos. Soc. 136 (2004), 269–286,<br>
            URL: <a href="https://www.cambridge.org/core/journals/mathematical-proceedings-of-the-cambridge-philosophical-society/article/abs/sl2-covariant-theory-of-genus-2-hyperelliptic-functions/2EE97A3BFFF5F70DE28F31560F0AC482" target="_blank"><em>https://www.cambridge.org/core/journals/mathematical-proceedings-of-the-cambridge-philosophical-society/article/abs/sl2-covariant-theory-of-genus-2-hyperelliptic-functions/2EE97A3BFFF5F70DE28F31560F0AC482</a>
        </li>
        <li>
            H. F. Baker, <em>An Introduction to the theory of multiply periodic functions</em>,(1907),<br>
        </li>
        <li>
            M. Hayashi, K. Shigemoto and T. Tsukioka, <em>The half-period addition formulae for genus two hyperelliptic ℘ functions and the Sp(4, R) Lie group structure</em>, J. Phys. Commun. 6 085004 (2022)<br>
            URL: <a href="https://iopscience.iop.org/article/10.1088/2399-6528/ac8521" target="_blank"><em>https://iopscience.iop.org/article/10.1088/2399-6528/ac8521</a>
        </li>
        <li>
            J. Bernatska, <em>Reality conditions for the KdV equation and exact quasi-periodic solutions in finite phase spaces</em>, Journal of Geometry and Physics, 206, 105322 (2024),<br>
            URL: <a href="https://arxiv.org/pdf/2312.10859" target="_blank"><em>https://arxiv.org/pdf/2312.10859</a>
        </li>
        <li>
            Enolski V.Z., Richter P.H.<em>Periods of hyperelliptic integrals expressed in terms of $\theta$-constants by means of Thomae formulae</em>, (2007),<br>
            URL: <a href="https://www.itp.uni-bremen.de/prichter/download/ThetaConst.pdf" target="_blank"><em>https://www.itp.uni-bremen.de/prichter/download/ThetaConst.pdf</a></li>
        <li>
            Julia Bernatska, <em>Abelian function fields on Jacobian varieties</em>, arXiv (2024),<br>
            URL: <a href="https://arxiv.org/abs/2412.05455" target="_blank">https://arxiv.org/abs/2412.05455</a> </li>
        <li>
            Julia Bernatska, <em>An analogue of Vélu's formulae in genus two</em>, arXiv (2024),<br>
            URL: <a href="https://arxiv.org/abs/2412.10284" target="_blank">https://arxiv.org/abs/2412.10284</a> </li>
        <li>
            Julia Bernatska, <em>Vector of Riemann constants</em>, Wolfram Community (2024),<br>
            URL: <a href="https://community.wolfram.com/groups/-/m/t/3296279" target="_blank">https://community.wolfram.com/groups/-/m/t/3296279</a> </li>

</ol>
</div>